In [5]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [1]:
from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection


In [2]:
g = traversal().withRemote(DriverRemoteConnection('ws://localhost:8182/gremlin','g'))

In [3]:
g.V().count()

[['V'], ['count'], ['values', '_ipython_canary_method_should_not_exist_'], ['values', '_ipython_canary_method_should_not_exist_']]

In [6]:
# Insert a new vertex
new_vertex = g.addV('person').property('name', 'John').property('age', 30).next()

# Insert another vertex
another_vertex = g.addV('person').property('name', 'Alice').property('age', 25).next()

# Insert an edge between the vertices
g.V(new_vertex).addE('knows').to(g.V(another_vertex)).next()

e[{'@type': 'janusgraph:RelationIdentifier', '@value': {'relationId': '3kn-3aw-4r9-6go'}}][4280-knows->8376]

In [13]:
g.V().valueMap(True).toList()

[{<T.id: 1>: 4208, <T.label: 4>: 'person', 'comName': ['value']},
 {<T.id: 1>: 4280, <T.label: 4>: 'person', 'name': ['John'], 'age': [30]},
 {<T.id: 1>: 8376, <T.label: 4>: 'person', 'name': ['Alice'], 'age': [25]}]

In [14]:
all_vertices = g.V().valueMap(True).toList()

# Print the nodes and their properties
for vertex_properties in all_vertices:
    print(vertex_properties)

{<T.id: 1>: 4208, <T.label: 4>: 'person', 'comName': ['value']}
{<T.id: 1>: 4280, <T.label: 4>: 'person', 'name': ['John'], 'age': [30]}
{<T.id: 1>: 8376, <T.label: 4>: 'person', 'name': ['Alice'], 'age': [25]}
